In [44]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField,IntegerType, StringType

In [95]:
filepath = '../../../data/bad_*.csv'
def get_schema():
    customeSchma = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("_corrupt_record", StringType(), True)
    ])
    
    return customeSchma
    
def read_csv_data(spark,filepath,error_mode = 'PERMISSIVE'):
    # error_mode = PERMISSIVE,DROPMALFORMED,FAILFAST
    dataframe = spark.read.format("csv").schema(get_schema()).\
    options(path=filepath,header=True,delimiter = ",").\
    option("mode",error_mode).\
    option("columnNameOfCorruptRecord", "_corrupt_record").load()
    return dataframe
    
def check_corrupted_records(daraframe)->int:
    error_data = dataframe.filter("_corrupt_record is not  null")
    return error_data.rdd.count()
    
def write_corrupted_records(daraframe,filepath):
    daraframe.write.format('json').mode('overwrite').option('path',filepath).save()

### Currupted records format
```
id,name,country,salary
1,khan,india,25000
2,mama,india,12500,
3,banan,canana,10101
4,banan,vancouver
```

In [4]:
spark = SparkSession.builder.master("local").appName("handle_corrupted_record").getOrCreate()

### Using PERMISSIVE MODE
When it meets the currupted records,puts the malformed string into a field configured by columnNameOfCorruptRecord, and sets malformed fields to null.

In [93]:
#https://spark.apache.org/docs/latest/sql-data-sources-csv.html
dataframe = read_csv_data(spark,filepath,error_mode = 'PERMISSIVE')
dataframe.show()

+---+-----+---------+------+-------------------+
| id| name|  country|salary|    _corrupt_record|
+---+-----+---------+------+-------------------+
|  1| khan|    india| 25000|               NULL|
|  2| mama|    india| 12500|2,mama,india,12500,|
|  3|banan|   canana| 10101|               NULL|
|  4|banan|vancouver|  NULL|  4,banan,vancouver|
+---+-----+---------+------+-------------------+



In [96]:
## Check if error is there, if yes store it into file
if check_corrupted_records(dataframe) > 0:
    print("currupted records found")
    bad_records_path = '../../../data/err/'
    write_corrupted_records(dataframe,bad_records_path)

    

currupted records found


### Using DROPMALFORMED MODE
ignores the whole corrupted records. This mode is unsupported in the CSV built-in functions

In [98]:
dataframe = read_csv_data(spark,filepath,error_mode = 'DROPMALFORMED')
dataframe.show()

+---+-----+-------+------+---------------+
| id| name|country|salary|_corrupt_record|
+---+-----+-------+------+---------------+
|  1| khan|  india| 25000|           NULL|
|  3|banan| canana| 10101|           NULL|
+---+-----+-------+------+---------------+



In [100]:
if check_corrupted_records(dataframe) > 0:
    print("currupted records found")
    bad_records_path = '../../../data/err/'
    write_corrupted_records(dataframe,bad_records_path)
else:
    print("No currupted records/Skipped currupted records")

No currupted records/Skipped currupted records


### Using FAILFAST MODE
throws an exception when it meets corrupted records

In [106]:
dataframe = read_csv_data(spark,filepath,error_mode = 'FAILFAST')
dataframe.show()
"""
Exception Message
[MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [2,mama,india,12500,2,mama,india,12500,].
Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
"""

Py4JJavaError: An error occurred while calling o803.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 118.0 failed 1 times, most recent failure: Lost task 0.0 in stage 118.0 (TID 102) (10.0.0.55 executor driver): org.apache.spark.SparkException: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [2,mama,india,12500,2,mama,india,12500,].
Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'. 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1610)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:79)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:456)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(It